In [ ]:
def xarray_to_tensor(dir_of_data):#the directory path
    timestamps_rea=[]
    list_data_forecast=[]
    list_tensor1=[]
    list_tensor2=[]
    dir_cams_forecast= os.listdir( dir_of_data)
    for i,x in enumerate(dir_cams_forecast):
        if x==".ipynb_checkpoints":
            continue

        #LIST_DATA
        data = xr.open_dataset(dir_of_data + x)
    #     data_vars=[x for x in data]
    #     missing_vars = [var for var in ['t', 'u', 'v', 'z'] if var not in data_vars]
    #     if missing_vars:
    #         continue
        list_vars_rea=[x for x in data]
        for i, var_name in enumerate(list_vars_rea):
            tensor=torch.tensor(data[var_name].values)
            tensor1, tensor2 = torch.split(tensor, 1, dim=0)
            list_tensor1.append(tensor1)
            list_tensor2.append(tensor2)

        data_vars1=torch.stack(list_tensor1)
        data_vars2=torch.stack(list_tensor2)
        list_tensor1=[]
        list_tensor2=[]
        data_vars1=F.interpolate(data_vars1,[3,54,81])
        data_vars2=F.interpolate(data_vars2,[3,54,81])
        list_data_forecast.append(data_vars1)
        list_data_forecast.append(data_vars2)

        #TIMESTAMP
        for x in data.time.values:
            timestamps_rea.append(x)

    timestamps_rea=pd.DatetimeIndex(timestamps_rea, tz="UTC")
    data_rea=torch.stack(list_data_forecast)
    return data_rea,timestamps_rea

In [ ]:
def check_shared_timestamps_idxs(cams_shared_time,shared_timestamps_labels,meta_data,timestamps_labels):
    correct=0
    for i in range(len(cams_shared_time)):
        meta_data["timestamp"][i]
        if meta_data["timestamp"][cams_shared_time[i]]==timestamps_labels[shared_timestamps_labels[i]]:
            correct+=1
    return f'Accuracy in percentages {correct//len(cams_shared_time)*100}'

In [ ]:
def sync_time(timestamps_lists,time=0):
    timestamps1, timestamps2 = timestamps_lists
    
    common_indices = [[],[]]
    common_dates = []
    
    for i, date1 in enumerate(timestamps1):
        mask = timestamps2 == date1+pd.Timedelta(hours=time)
        if np.any(mask):
            j = np.where(mask)[0][0]
            common_indices[0].append(i)
            common_indices[1].append(j)
            #common_indices.append((i, j))
            common_dates.append(date1)
    common_dates=pd.to_datetime(common_dates)
    
    return common_indices, common_dates

In [ ]:
def matching_vars_forecast_rea(meta_data_fore,meta_data_rea,data_fore,data_rea):
    fore_index_vars=[i for i,x in enumerate(meta_data_fore['vars']) if x in meta_data_rea['vars']]
    print(fore_index_vars)
    rea_index_vars=[meta_data_rea["vars"].index(x) for x in meta_data_fore["vars"] if x in meta_data_rea["vars"]]
    print(rea_index_vars)
    fore_vars=[x for i,x in enumerate(meta_data_fore['vars']) if x in meta_data_rea['vars']]
    meta_data_rea['vars']=fore_vars
    meta_data_fore['vars']=fore_vars
    
    return data_fore[:,fore_index_vars],data_rea[:,rea_index_vars]

In [ ]:
def plot_cams_vs_insitu_stations(cams_data,insitu_data,selected_leadtimes:list,stations_names:list,vertical_line_dates=None):
    for station in stations_names:
        print(f'the station - {station}')
        index_station_cams=np.where(meta_data_3d['station_id_name']==station)[0]
        index_station_dust=np.where(station_dust==station)[0]
        print(index_station_dust)
        for lead_time1 in selected_leadtimes:   
            if lead_time1=='0':
                tensor1 = cams_data[index_station_cams[0],:, int(int(lead_time1)/3)]
                tensor2 = labels[:,index_station_dust[0],0]  
            else:
                index_lead_time = np.where(np.array(lead_time_list_dust) ==lead_time1 )[0]
                print(index_lead_time)
                tensor1 = cams_data[index_station_cams[0],:, int(int(lead_time1)/3)]
                tensor2 = labels[:,index_station_dust[0],index_lead_time[0]]

            # Create traces for the two tensors
            trace1 = go.Scatter(x=meta_data_3d['timestamp'][shared_timestamps_idxs[0]], y=tensor1, mode='lines', name=f'Station_cams {station} ')
            trace2 = go.Scatter(x=meta_data_3d['timestamp'][shared_timestamps_idxs[0]], y=tensor2, mode='lines', name=f'Station_insitu {station}')

            # Create a layout for the graph
            layout = go.Layout(title=f'Comparison of Leadtime {lead_time1} hours for station {station} ',
                           xaxis=dict(title='timestamps'),
                           yaxis=dict(title='value'))
            shapes = []
            for date in vertical_line_dates:
                shapes.append(dict(type='line', x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper', line=dict(color='green', width=1, dash='dash')))
            layout['shapes'] = shapes

            # Combine the traces and layout to create a figure
            fig = go.Figure(data=[trace1, trace2], layout=layout)

            # Display the graph in Jupyter Notebook or an HTML file
            pyo.iplot(fig)  # To display in Jupyter Notebook

In [ ]:
import gradio as gr
import joblib

# Load your trained model (assuming you have saved it as 'model.pkl')
# model = joblib.load('model.pkl')

# Define your prediction function
def ui_for_geneator_images()
    def predict_price(airline, source_city, stops, destination_city, flight_class, days_left):
        # Convert inputs to the expected format
        input_data = [[airline, source_city, stops, destination_city, flight_class, days_left]]

        # Predict the price
        # predicted_price = grad.predict(input_data)[0]
        return 1000

    # Define the UI components using the updated gradio components
    iface = gr.Interface(
        fn=predict_price,
        inputs=[
            gr.Dropdown(choices=[1, 2, 3, 4], label="Airline"),
            gr.Dropdown(choices=[0, 1, 2, 3, 4, 5], label="Source City"),
            gr.Dropdown(choices=[0, 1, 2], label="Stops"),
            gr.Dropdown(choices=[0, 1, 2, 3, 4], label="Destination City"),
            gr.Radio(choices=[0, 1], label="Flight Class"),
            gr.Slider(minimum=0, maximum=365, label="Days Left")
        ],
        outputs="number",
        title="Flight Price Predictor",
        description="Enter flight details to get the predicted price."
    )

    # Launch the UI
    iface.launch()